# M14a - Multimodales RAG Enhanced

Dieses Notebook zeigt die Verwendung der erweiterten RAG-Funktionen mit automatischen Bildbeschreibungen.

**Neue Features:**
- Automatische Bildbeschreibungen mit GPT-4o-mini
- Cross-References zwischen Text und Bildern
- Cross-Modal-Retrieval
- Hybride Suche über Text und Bildbeschreibungen

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

In [ ]:
#@title 📂 Dokumente & Bilder kopieren { display-mode: "form" }
!rm -rf files
!mkdir files

# --- Texte
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/roboter.txt -o files/roboter.txt

# --- Bilder
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg -o files/a_retro-futuristic_robot_dall_e.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/hedra_cyborg.png -o files/hedra_cyborg.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/apfel.jpg -o files/apfel.jpg

## 1. Setup
Zuerst führen wir das Original-Notebook M14 bis Sektion 2 aus.

In [ ]:
%run M14_Multimodal_RAG.ipynb

## 2. Enhanced Funktionen laden

In [ ]:
from M14a_enhanced_rag import *

## 3. Enhanced RAG-System initialisieren

In [ ]:
# Initialisiere mit Vision-LLM Support
rag_components = init_enhanced_rag_system()

## 4. Bildbeschreibung testen

In [ ]:
# Teste Bildbeschreibung für ein einzelnes Bild
test_image = './files/apfel.jpg'

if Path(test_image).exists():
    description = generate_image_description(rag_components.vision_llm, test_image)
    print(f'Beschreibung: {description}')
else:
    print(f'Bild nicht gefunden: {test_image}')

## 5. Verzeichnis mit automatischen Bildbeschreibungen verarbeiten

In [ ]:
# Verarbeite alle Dateien mit automatischen Bildbeschreibungen
results = process_directory_enhanced(
    rag_components,
    './files',
    include_images=True,
    auto_describe_images=True
)

print(f'Text-Dokumente: {results["texts"]}')
print(f'Bilder: {results["images"]}')
print(f'Bildbeschreibungen: {results["image_descriptions"]}')

## 6. Erweiterte Textsuche (inkl. Bildbeschreibungen)

In [ ]:
# Textsuche findet nun auch Bildbeschreibungen
query = 'Roboter'
result = search_texts_enhanced(rag_components, query, k=5)
print(result)

## 7. Multimodale Suche mit Cross-Modal-Retrieval

In [ ]:
# Kombinierte Suche über alle Modalitäten
queries = ['Roboter', 'Cyborg', 'Apfel']

for query in queries:
    print(f'
=== Query: {query} ===')
    result = multimodal_search_enhanced(
        rag_components,
        query,
        k_text=3,
        k_images=3,
        enable_cross_modal=True
    )
    print(result)

## 8. Cross-Reference-Analyse

In [ ]:
# Analysiere Cross-References zwischen Collections
all_texts = rag_components.text_collection.get()
image_descriptions = [
    (id, meta) for id, meta in zip(all_texts['ids'], all_texts['metadatas'])
    if meta.get('doc_type') == 'image_description'
]

print(f'Gefundene Bildbeschreibungen: {len(image_descriptions)}')

for text_id, meta in image_descriptions[:3]:
    print(f"
{meta.get('filename', 'Unbekannt')}:")
    print(f"  Text-ID: {text_id}")
    print(f"  Bild-ID: {meta.get('image_doc_id', 'N/A')}")
    print(f"  Beschreibung: {meta.get('description', 'N/A')[:100]}...")

## 9. Vergleich: Original vs. Enhanced

### Vorteile der Enhanced Version:

1. **Semantische Bildsuche**: Textanfragen finden automatisch relevante Bilder
2. **Bessere Trefferquote**: Bildbeschreibungen erhöhen Recall
3. **Einheitliche Suche**: Eine Query durchsucht Text und Bilder
4. **Explainable AI**: Beschreibungen machen Ergebnisse nachvollziehbar
5. **Cross-Modal Intelligence**: System versteht Zusammenhänge